# Libs 

In [2]:
import urllib.request
import urllib.parse
import pandas as pd
import numpy as np
import ast
from haversine import haversine

# Data

In [3]:
f = open("files/keys.txt", 'r')
Api_Key = f.readline()
f.close()

도로명 주소 이용

In [20]:
#성균관대주소, 혜화역 주소, 한성대역 주소
SKKU_Address = '종로구 성균관로 25-2'
St_Hyehwa_Address = '종로구 대학로 122' #'대학로 지하 120'
St_Hansung_Address = '성북구 삼선교로 지하 1'

In [5]:
df = pd.read_csv('files/Merged_data.csv', encoding='CP949')
df = df.reindex(columns = df.columns.tolist() + ["위도","경도",'학교까지_거리','혜화역까지_거리','한성대역까지_거리'])

# From Address to Longitude&Latitude

In [6]:
def Find_Longitude_Latitude(address):
  apiUrl = 'http://api.vworld.kr/req/address?service=address&request=getCoord&key='+Api_Key+'&'
  values = { 'address':address, 'type':'ROAD' }
  param = urllib.parse.urlencode(values)
  Adding = apiUrl+param

#   headers = {'User-Agent' : 'Chrome/87.0.4280.88'}
  req = urllib.request.Request(Adding)
  res = urllib.request.urlopen(req)

  respon_data = res.read().decode()

  DataDict = ast.literal_eval(respon_data)
  
  v_check = DataDict['response']['status']

  if v_check == 'NOT_FOUND' :
    return np.nan, np.nan
  
  else:
    longtitude = DataDict['response']['result']['point']['x']
    latitude = DataDict['response']['result']['point']['y']

    return latitude, longtitude

In [17]:
#3 point value
loc_SKKU = tuple(map(float, Find_Longitude_Latitude(SKKU_Address)))
loc_Hyehwa_Stn = tuple(map(float, Find_Longitude_Latitude(St_Hyehwa_Address)))
loc_Hansung_Stn = tuple(map(float, Find_Longitude_Latitude(St_Hansung_Address)))

In [11]:
for i in range(len(df)):
    df.loc[i, '위도'], df.loc[i, '경도'] = tuple(map(float, Find_Longitude_Latitude(df.loc[i, '도로명'])))

# From Longitude&Latitude to Distance

In [18]:
for i in range(len(df)):
    start = (float(df.loc[i, '위도']), float(df.loc[i, '경도']))
    df.loc[i, '학교까지_거리'] = haversine(start, loc_SKKU, unit = 'm') # (lat, lon)
    df.loc[i, '혜화역까지_거리'] = haversine(start, loc_Hyehwa_Stn, unit = 'm')
    df.loc[i, '한성대역까지_거리'] = haversine(start, loc_Hansung_Stn, unit = 'm')

In [19]:
df.head(10)

,건물명,건축년도,계약년월,계약일,단지명,도로명,지번,보증금(만원),보증금->월세,본번,...,건물식별번호,건물연령,공동주택명,공시가격,데이터기준일자,위도,경도,학교까지_거리,혜화역까지_거리,한성대역까지_거리
0,NaN,1999,201804,13,동성아파트(1동),동숭4라길 29,4-161,20000.0,66.666667,4,...,24809.0,22.0,동성아파트(1동),162000000.0,2020-07-27,37.583375,127.006271,1235.880709,378.810239,559.530861
1,NaN,1999,201910,18,동성아파트(2동),동숭4라길 27,4-157,22000.0,73.333333,4,...,29649.0,22.0,동성아파트(2동),158000000.0,2020-07-27,37.583500,127.006149,1221.044377,373.985483,545.305534
2,(129-23),1997,201806,2,NaN,동숭4나길 28-1,129-23,13000.0,43.333333,129,...,17641.0,24.0,(129-23),122000000.0,2020-07-27,37.583036,127.005743,1206.473960,322.626687,597.085546
3,(129-24),1997,201712,9,NaN,동숭4나길 28,129-24,14000.0,46.666667,129,...,22280.0,24.0,(129-24),98500000.0,2020-07-27,37.583045,127.005632,1197.013260,313.528991,596.535653
4,(129-74),1994,201802,20,NaN,낙산1길 22-1,129-74,25000.0,83.333333,129,...,10769.0,27.0,(129-74),136000000.0,2020-07-27,37.581118,127.005861,1312.654556,339.899030,810.031517
5,(129-74),1994,201803,26,NaN,낙산1길 22-1,129-74,20000.0,66.666667,129,...,10769.0,27.0,(129-74),80700000.0,2020-07-27,37.581118,127.005861,1312.654556,339.899030,810.031517
6,(129-74),1994,201807,28,NaN,낙산1길 22-1,129-74,33000.0,110.000000,129,...,10769.0,27.0,(129-74),136000000.0,2020-07-27,37.581118,127.005861,1312.654556,339.899030,810.031517
7,(130-7),2014,201806,27,NaN,동숭2길 13-3,130-7,1000.0,48.333333,130,...,NaN,NaN,(130-7),61400000.0,2020-07-27,37.581068,127.005069,1256.475426,277.959329,819.514180
8,(14-2),1998,201801,24,NaN,동숭4나길 5,14-2,7000.0,48.333333,14,...,10902.0,23.0,(14-2),62500000.0,2020-07-27,37.583173,127.004496,1099.579313,226.983207,596.110922
9,(14-2),1998,201808,2,NaN,동숭4나길 5,14-2,9000.0,40.000000,14,...,10902.0,23.0,(14-2),62500000.0,2020-07-27,37.583173,127.004496,1099.579313,226.983207,596.110922


In [21]:
df.to_csv('tmp.csv', index=False, encoding='cp949')

# Reference
https://stricky.tistory.com/283
